# Tutorial 8: Visualization

## Introduction

When you are running a simulation, it is often useful to see what is going on
by visualizing particles in a 3D view or by plotting observables over time.
That way, you can easily determine things like whether your choice of parameters
has led to a stable simulation or whether your system has equilibrated. You may
even be able to do your complete data analysis in real time as the simulation progresses.

Thanks to **ESPResSo**'s Python interface, we can make use of standard libraries
like Mayavi or OpenGL (for interactive 3D views) and Matplotlib (for line graphs)
for this purpose. We will also use NumPy, which both of these libraries depend on,
to store data and perform some basic analysis.

## Simulation

First, we need to set up a simulation.
We simulate a simple Lennard-Jones liquid in this tutorial using the script <tt>/doc/tutorials/08-visualization/scripts/simulation.py</tt>.

## Live plotting

Let's have a look at the total energy of the simulation.
We can determine the individual energies in the system using

In [ ]:
print (system.analysis.energy())

to get

In [ ]:
OrderedDict([
        ('total', 1840.118038871784),
        ('ideal', 1358.743742464325),
        ('bonded', 0.0), 
        (('nonBonded', 0, 0), 481.374296407459), 
        ('nonBonded', 481.374296407459), 
        ('coulomb', 0.0)])

Write that command right after the call to <tt>main()</tt> and see if you can get a similar result.
Now we want to store the total energy over time in a NumPy array.
To do that, modify the <tt>main</tt> function definition to be the following:

In [ ]:
energies = numpy.empty((int_steps,2))
def main():
    for i in range(0, int_n_times):
        print("run %d at time=%f " % (i, system.time))
        system.integrator.run(int_steps)
        energies[i] = (system.time, system.analysis.energy()['total'])

Now we can do some analysis on the stored energies.
For example, let us calculate the time-averaged energy:

In [ ]:
print ("Average energy: %.6g" % energies[:,1].mean())

We can also plot the energy over time by adding

In [ ]:
pyplot.xlabel("time")
pyplot.ylabel("energy")
pyplot.plot(energies[:,0],energies[:,1])
pyplot.show()

to the end of the script. Of course, this plot only gets shown after the entire simulation is completed.
To get an interactive plot, we update it from within the integration loop.

In [ ]:
energies = numpy.empty((int_steps,2))
current_time = -1
pyplot.xlabel("time")
pyplot.ylabel("energy")
plot, = pyplot.plot([0],[0])
pyplot.show(block=False)
def update_plot():
    if current_time < 0:
        return
    i = current_time
    plot.set_xdata(energies[:i+1,0])
    plot.set_ydata(energies[:i+1,1])
    pyplot.xlim(0, energies[i,0])
    pyplot.ylim(energies[:i+1,1].min(), energies[:i+1,1].max())
    pyplot.draw()
    pyplot.pause(0.01)
def main():
    global current_time
    for i in range(0, int_n_times):
        print("run %d at time=%f " % (i, system.time))
        system.integrator.run(int_steps)
        energies[i] = (system.time, system.analysis.energy()['total'])
        current_time = i
        update_plot()

One shortcoming of this simple method is that one cannot interact with the controls of the plot window (e.g. resize the window or zoom around in the graph).
This will be resolved using multiple threads when we combine the plotting with the 3D visualization in the next section.

## Live visualization

In order to be able to interact with the live visualization, we need to move the main integration loop into a secondary thread and run the visualization in the main thread (note that visualization or plotting cannot be run in secondary threads). First, let's revert to the main loop without plotting:

In [ ]:
energies = numpy.empty((int_steps,2))
def main():
    for i in range(0, int_n_times):
        print("run %d at time=%f " % (i, system.time))
        system.integrator.run(int_steps)
        energies[i] = (system.time, system.analysis.energy()['total'])

Then, add the following line after the particle setup code:

In [ ]:
visualizer = visualization.openGLLive(system)

Now, go to the end of the <tt>main</tt> function definition and add

In [ ]:
visualizer.update()

which sends the current simulation state to the visualizer.
Now, go to the line where <tt>main()</tt> is called and replace it with the following code, which dispatches the function in a secondary thread, and then opens the visualizer window:

In [ ]:
t = Thread(target=main)
t.daemon = True
t.start()
visualizer.start()

To follow the trajectories, try decreasing the integration steps to 1 and the
time step to 0.001.  While the simulation is running, you can move and zoom
around with your mouse. Alternatively, you can try mayavi by
switching the visualizer to:

In [ ]:
visualizer = visualization.mayaviLive(system)

In mayavi, explore the buttons in the toolbar to see how the graphical representation can be changed

## Combined live visualization and plotting

Now let's merge the code from the preceding two sections so we can see the energy graph while viewing the 3D visualization of the particles.
Do do that, we copy the <tt>pyplot</tt>-related lines from above:

In [ ]:
current_time = -1
pyplot.xlabel("time")
pyplot.ylabel("energy")
plot, = pyplot.plot([0],[0])
pyplot.show(block=False)
def update_plot():
    if current_time < 0:
        return
    i = current_time
    plot.set_xdata(energies[:i+1,0])
    plot.set_ydata(energies[:i+1,1])
    pyplot.xlim(0, energies[i,0])
    pyplot.ylim(energies[:i+1,1].min(), energies[:i+1,1].max())
    pyplot.draw()
    pyplot.pause(0.01)

Then we merge the <tt>main</tt> function definitions from both the previous sections.

In [ ]:
def main():
    global current_time
    for i in range(0, int_n_times):
        print("run %d at time=%f " % (i, system.time))
        system.integrator.run(int_steps)
        energies[i] = (system.time, system.analysis.energy()['total'])
        current_time = i
        visualizer.update()
        # update_plot() cannot be called from here

However, as we now have multiple threads, we cannot simply call <tt>update_plot()</tt> from the <tt>main</tt> function definition.
Instead, we register it as a callback with the visualizer before we start up the visualizer GUI:

In [ ]:
t = Thread(target=main)
t.daemon = True
t.start()
visualizer.register_callback(update_plot, interval=500)
visualizer.start()

## Customizing the OpenGL visualizer

Visualization of more advanced features of **ESPResSo** is also possible (e.g.
bonds, constraints, Lattice Boltzmann) with the OpenGL
visualizer. There are a number of optional keywords that can be used to specify the
appearance of the visualization, they are simply stated after
 <tt>system</tt> when creating the visualizer instance.
See the following examples:

In [ ]:
# Enables particle dragging via mouse:
visualizer = visualization.openGLLive(system, drag_enabled=True)

# Use a white background:
visualizer = visualization.openGLLive(system, background_color = [1,1,1])

# Use red color for all (uncharged) particles
visualizer = visualization.openGLLive(system, particle_type_colors = [[1,0,0]])

The visualizer options are stored in the dict <tt>visualizer.specs</tt>,
the following snippet prints out the current configuration nicely: 

In [ ]:
for k in sorted(visualizer.specs.keys(), key=lambda s: s.lower()): print("{:30}  {}".format(k, visualizer.specs[k]))

All keywords are explained in the Online Documentation at
http://espressomd.org/html/doc/visualization.html#opengl-visualizer.
Specific visualization examples for **ESPResSo** can be found in the <tt>samples</tt>
folder. You may need to recompile **ESPResSo** with the required features used in the
examples.